In [180]:
import pandas as pd
import random
from sklearn.neighbors import KNeighborsClassifier

In [181]:
#Hasta 15 de Abril (para entrenar)

postulantes_edu_df = pd.read_csv('fiuba_1_postulantes_educacion_h15.csv')
postulantes_gye_df = pd.read_csv('fiuba_2_postulantes_genero_y_edad_h15.csv')
vistas_df = pd.read_csv('fiuba_3_vistas_h15.csv')
postulaciones_df = pd.read_csv('fiuba_4_postulaciones_h15.csv')
avisos_online_df = pd.read_csv('fiuba_5_avisos_online_h15.csv')
avisos_detalle_df = pd.read_csv('fiuba_6_avisos_detalle_h15.csv')

In [182]:
#Originales TP1 (para entrenar)

postulantes_edu_df2 = pd.read_csv('fiuba_1_postulantes_educacion.csv')
postulantes_gye_df2 = pd.read_csv('fiuba_2_postulantes_genero_y_edad.csv')
vistas_df2 = pd.read_csv('fiuba_3_vistas.csv')
postulaciones_df2 = pd.read_csv('fiuba_4_postulaciones.csv')
avisos_online_df2 = pd.read_csv('fiuba_5_avisos_online.csv')
avisos_detalle_df2 = pd.read_csv('fiuba_6_avisos_detalle.csv')

In [183]:
#Realizo el Merge del tipo outer de los datos
#Originales TP1 y Datos hasta 15 de abril

postulantes_edu_df = postulantes_edu_df.merge(postulantes_edu_df2,how='outer')
postulantes_gye_df = postulantes_gye_df.merge(postulantes_gye_df2,how='outer')
vistas_df = vistas_df.merge(vistas_df2,how='outer')
postulaciones_df = postulaciones_df.merge(postulaciones_df2,how='outer')
avisos_online_df = avisos_online_df.merge(avisos_online_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_df2,how='outer')

In [184]:
#Desde 15 de abril (para entrenar)

postulantes_edu_df2 = pd.read_csv('fiuba_1_postulantes_educacion_d15.csv')
postulantes_gye_df2 = pd.read_csv('fiuba_2_postulantes_genero_y_edad_d15.csv')
vistas_df2 = pd.read_csv('fiuba_3_vistas_d15.csv')
avisos_detalle_df2 = pd.read_csv('fiuba_6_avisos_detalle_d15.csv')

In [185]:
#Avisos detalle que faltaban
avisos_detalle_missing = pd.read_csv('fiuba_6_avisos_detalle_missing_nivel_laboral.csv')

In [186]:
#Mergeamos todo

postulantes_edu_df = postulantes_edu_df.merge(postulantes_edu_df2,how='outer')
postulantes_gye_df = postulantes_gye_df.merge(postulantes_gye_df2,how='outer')
vistas_df = vistas_df.merge(vistas_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_df2,how='outer')
avisos_detalle_df = avisos_detalle_df.merge(avisos_detalle_missing,how='outer')

# Libero Espacio de la Memoria

In [187]:
postulantes_edu_df2 = 0
postulantes_gye_df2 = 0
vistas_df2 = 0
avisos_detalle_df2 = 0

# Filtrado de Postulantes con informacion completa

In [188]:
postulantes_edu_df.head()

,idpostulante,nombre,estado
0,ZjlZ,Master,En Curso
1,NdJl,Posgrado,En Curso
2,5kNq,Otro,En Curso
3,8rYD,Master,En Curso
4,1Wvj,Universitario,En Curso


In [189]:
postulantes_gye_df.head()

,idpostulante,fechanacimiento,sexo
0,6MM,1985-01-01,MASC
1,Nzz,NaN,NO_DECLARA
2,ZX1,NaN,NO_DECLARA
3,Nq5,NaN,NO_DECLARA
4,ebE,1952-07-07,MASC


In [190]:
print(postulantes_edu_df.shape)
print(postulantes_gye_df.shape)

(693641, 3)
(505382, 3)


In [191]:
print(postulantes_edu_df.idpostulante.unique().shape)
print(postulantes_gye_df.idpostulante.unique().shape)

(447909,)
(504407,)


In [192]:
#Ojo porque hay que ver el tema del secundario y eso. 
#Haciendo este Merge tenemos los usuarios que sabemos su genero edad y educacion.
postulantes_filtrados_df = postulantes_gye_df.merge(postulantes_edu_df,on='idpostulante',how='outer')

print(postulantes_filtrados_df.shape)

postulantes_filtrados_df['idpostulante'].unique().shape

(751563, 5)


(504407,)

In [193]:
postulantes_filtrados_df.head()

,idpostulante,fechanacimiento,sexo,nombre,estado
0,6MM,1985-01-01,MASC,NaN,NaN
1,Nzz,NaN,NO_DECLARA,NaN,NaN
2,ZX1,NaN,NO_DECLARA,NaN,NaN
3,Nq5,NaN,NO_DECLARA,NaN,NaN
4,ebE,1952-07-07,MASC,Universitario,Graduado


In [199]:
postulantes_filtrados_df['nombre'].fillna('NO_DECLARA',inplace=True)
postulantes_filtrados_df['estado'].fillna('NO_DECLARA', inplace=True)
postulantes_filtrados_df['fechanacimiento'].fillna('1918-01-01', inplace=True)

In [200]:
postulantes_filtrados_df.head()

,idpostulante,fechanacimiento,sexo,nombre,estado
0,6MM,1985-01-01,MASC,NO_DECLARA,NO_DECLARA
1,Nzz,1918-01-01,NO_DECLARA,NO_DECLARA,NO_DECLARA
2,ZX1,1918-01-01,NO_DECLARA,NO_DECLARA,NO_DECLARA
3,Nq5,1918-01-01,NO_DECLARA,NO_DECLARA,NO_DECLARA
4,ebE,1952-07-07,MASC,Universitario,Graduado


In [201]:
#Sacamos los repetidos de postulantes_filtrados_df
#Aca NO se tuvo en cuenta si me quedo con el usuario que hizo Master o Secundario, etc.

postulantes_filtrados_df = postulantes_filtrados_df.drop_duplicates('idpostulante')
postulantes_filtrados_df.shape

(504407, 5)

In [202]:
postulantes_filtrados_df['estado'].value_counts()

Graduado      257790
En Curso      169432
NO_DECLARA     56498
Abandonado     20687
Name: estado, dtype: int64

In [203]:
#Convierto las variables categoricas a variables numericas para poder procesarlas con el algoritmo.
postulantes_filtrados_df['sexo'] = pd.Categorical(postulantes_filtrados_df['sexo'])
postulantes_filtrados_df['sexo'] = postulantes_filtrados_df['sexo'].cat.codes
postulantes_filtrados_df['sexo'].value_counts()

1    250962
2    227653
3     25785
0         7
Name: sexo, dtype: int64

In [204]:
#Convierto las variables categoricas a variables numericas para poder procesarlas con el algoritmo.
postulantes_filtrados_df['nombre'] = pd.Categorical(postulantes_filtrados_df['nombre'])
postulantes_filtrados_df['nombre'] = postulantes_filtrados_df['nombre'].cat.codes
postulantes_filtrados_df['nombre'].value_counts()

7    189432
5    164472
6     66101
2     56498
3     14344
4      7939
1      5286
0       335
Name: nombre, dtype: int64

In [205]:
#Convierto las variables categoricas a variables numericas para poder procesarlas con el algoritmo.
postulantes_filtrados_df['estado'] = pd.Categorical(postulantes_filtrados_df['estado'])
postulantes_filtrados_df['estado'] = postulantes_filtrados_df['estado'].cat.codes
postulantes_filtrados_df['estado'].value_counts()

2    257790
1    169432
3     56498
0     20687
Name: estado, dtype: int64

In [206]:
#Obtengo la edad
postulantes_filtrados_df = postulantes_filtrados_df[ (postulantes_filtrados_df['fechanacimiento'].str.startswith('0')) == False]
postulantes_filtrados_df.loc[:,'fechanacimiento'] = pd.to_datetime(postulantes_filtrados_df['fechanacimiento']) 
postulantes_filtrados_df.loc[:,'edad'] = (2018 - postulantes_filtrados_df.fechanacimiento.dt.year)
postulantes_filtrados_df.drop(columns='fechanacimiento',axis=1,inplace=True)
postulantes_filtrados_df.head()
#Generar edad por bloques, como queremos hacer con las empresas mas populares.

,idpostulante,sexo,nombre,estado,edad
0,6MM,2,2,3,33
1,Nzz,3,2,3,100
2,ZX1,3,2,3,100
3,Nq5,3,2,3,100
4,ebE,2,7,2,66


# Avisos Detalle

In [207]:
avisos_detalle_df.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,1111556097,1,Responsable de Marketing-Producto Div. Constru...,<p>Buscamos un <strong>Responsable de Producto...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.
1,1111949392,1,Asesor de Servicios - Postventa Chevrolet,"<p><span style="""">Para nuestro cliente, conces...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan
2,1112145935,1,Analistas de Impuestos Sr Ssr y Jr,<p>Importante Estudio de Auditoria y Consultor...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart
3,1112146010,1,Software Engineer,<p> </p><p>Chevron is a multinational company ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center
4,1112211475,1,Business Developer Manager,"<p>Importante empresa de energías renovables, ...",Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,RANDSTAD


In [208]:
print(avisos_detalle_df.shape)
print(avisos_detalle_df['idaviso'].unique().shape)

(25901, 11)
(25288,)


In [209]:
#Sacamos los repetidos de avisos_detalle
avisos_detalle_df = avisos_detalle_df.drop_duplicates('idaviso')
avisos_detalle_df.shape

(25288, 11)

In [210]:
#Me quedo con los avisos del archivo de postulaciones. Esto seria, solo los avisos que tuvieron postulaciones.


#¿Aca me podria quedar con los archivos que NO tuvieron postulaciones para armar un set de NO postulaciones?

column = postulaciones_df['idaviso'].to_frame()
avisos_detalle_filtrados_df = column.drop_duplicates('idaviso').merge(avisos_detalle_df,on='idaviso',how='inner')
#avisos_no_postulaciones_df = pd.merge(avisos_detalle_df,avisos_detalle_filtrados_df,on='idaviso',how='right') 
#IMPLEMENTAR
avisos_detalle_filtrados_df.shape

(18695, 11)

In [153]:
#avisos_no_postulaciones_df.shape


In [212]:
#avisos_detalle_df.shape

Hago la conversion de variables categoricas a variables numericas

In [213]:
#Hago un drop de las columnas que no voy a utilizar.
#Mas adelanto debo modificar esto porque la idea es usar denominacion empresa, descripcion y titulo para nuevos features.

avisos_detalle_filtrados_df= avisos_detalle_filtrados_df.drop(columns=['denominacion_empresa','idpais','mapacalle','ciudad','descripcion','titulo'],axis=1)


In [214]:
avisos_detalle_filtrados_df['nivel_laboral'] = pd.Categorical(avisos_detalle_filtrados_df['nivel_laboral'])
avisos_detalle_filtrados_df['nivel_laboral'] = avisos_detalle_filtrados_df['nivel_laboral'].cat.codes
avisos_detalle_filtrados_df['nivel_laboral'].value_counts()

 4    12779
 2     2922
 3     1266
 1     1161
-1      330
 0      237
Name: nivel_laboral, dtype: int64

In [215]:
avisos_detalle_filtrados_df['nombre_area'] = pd.Categorical(avisos_detalle_filtrados_df['nombre_area'])
avisos_detalle_filtrados_df['nombre_area'] = avisos_detalle_filtrados_df['nombre_area'].cat.codes
avisos_detalle_filtrados_df['nombre_area'].value_counts()

183    2297
29     1338
2      1277
141    1178
143     830
41      590
168     579
116     442
14      439
129     299
80      293
115     289
152     287
166     284
5       281
118     236
33      228
25      223
4       211
117     205
135     195
179     195
77      193
45      192
151     189
50      172
172     170
173     168
113     165
121     156
       ... 
37        3
175       3
67        3
126       2
148       2
18        2
71        2
105       2
124       2
137       2
111       2
55        2
10        2
65        2
74        1
106       1
68        1
181       1
128       1
171       1
147       1
176       1
119       1
28        1
51        1
79        1
178       1
47        1
155       1
83        1
Name: nombre_area, Length: 186, dtype: int64

In [216]:
avisos_detalle_filtrados_df['nombre_area'] = pd.Categorical(avisos_detalle_filtrados_df['nombre_area'])
avisos_detalle_filtrados_df['nombre_area'] = avisos_detalle_filtrados_df['nombre_area'].cat.codes
avisos_detalle_filtrados_df['nombre_area'].value_counts()

183    2297
29     1338
2      1277
141    1178
143     830
41      590
168     579
116     442
14      439
129     299
80      293
115     289
152     287
166     284
5       281
118     236
33      228
25      223
4       211
117     205
135     195
179     195
77      193
45      192
151     189
50      172
172     170
173     168
113     165
121     156
       ... 
37        3
175       3
67        3
126       2
148       2
18        2
71        2
105       2
124       2
137       2
111       2
55        2
10        2
65        2
74        1
106       1
68        1
181       1
128       1
171       1
147       1
176       1
119       1
28        1
51        1
79        1
178       1
47        1
155       1
83        1
Name: nombre_area, Length: 186, dtype: int64

In [217]:
avisos_detalle_filtrados_df['tipo_de_trabajo'] = pd.Categorical(avisos_detalle_filtrados_df['tipo_de_trabajo'])
avisos_detalle_filtrados_df['tipo_de_trabajo'] = avisos_detalle_filtrados_df['tipo_de_trabajo'].cat.codes
avisos_detalle_filtrados_df['tipo_de_trabajo'].value_counts()

1    17127
2     1090
7      179
5      105
8       60
3       59
4       53
0       18
6        3
9        1
Name: tipo_de_trabajo, dtype: int64

In [218]:
avisos_detalle_filtrados_df['nombre_zona'] = pd.Categorical(avisos_detalle_filtrados_df['nombre_zona'])
avisos_detalle_filtrados_df['nombre_zona'] = avisos_detalle_filtrados_df['nombre_zona'].cat.codes
avisos_detalle_filtrados_df['nombre_zona'].value_counts()

5    17126
1     1543
0       15
4        9
3        1
2        1
Name: nombre_zona, dtype: int64

In [219]:
avisos_detalle_filtrados_df.head()

,idaviso,nombre_zona,tipo_de_trabajo,nivel_laboral,nombre_area
0,1112248724,5,1,4,118
1,1112286523,5,1,4,157
2,1112272060,5,1,4,151
3,1112288401,5,1,4,151
4,1112300563,5,1,4,157


# Postulaciones

In [220]:
postulantes_filtrados_df['idpostulante'].unique().shape

(504372,)

In [221]:
postulaciones_df.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


In [222]:
#Comparamos los "usuarios" con los  postulantes.

print(postulantes_filtrados_df.idpostulante.unique().shape)
print(postulaciones_df.idpostulante.unique().shape)

(504372,)
(348883,)


In [223]:
#Como resultado tenemos los postulantes que coinciden en postulaciones y postulantes
#Observamos que hay postulaciones de usuarios que no estan en el dataframe de postulantes.
postulantes_filtrados_efectivos_df = postulaciones_df.merge(postulantes_filtrados_df,on='idpostulante',how='inner')
postulantes_filtrados_efectivos_df.idpostulante.unique().shape

(348864,)

In [224]:
postulantes_filtrados_efectivos_df = postulantes_filtrados_efectivos_df.drop_duplicates('idpostulante')
postulantes_filtrados_efectivos_df.shape

(348864, 7)

In [225]:
postulantes_filtrados_efectivos_df =  postulantes_filtrados_efectivos_df.drop(['idaviso','fechapostulacion'],axis='columns')
postulantes_filtrados_efectivos_df = postulantes_filtrados_efectivos_df.reset_index()
postulantes_filtrados_efectivos_df =  postulantes_filtrados_efectivos_df.drop('index',axis='columns')
postulantes_filtrados_efectivos_df.head()

,idpostulante,sexo,nombre,estado,edad
0,NjlD,1,7,2,37
1,ZaO5,1,6,2,40
2,NdJl,2,4,1,49
3,5kNq,2,3,1,45
4,Ez8J,2,7,0,50


In [226]:
postulantes_filtrados_efectivos_df['nombre'].value_counts()

7    136289
5    120222
6     47565
2     25190
3     10633
4      5280
1      3454
0       231
Name: nombre, dtype: int64

In [227]:
# postulantes_filtrados_efectivos_df tiene los postulantes que efectivamente se postularon
# y que ademas tienen informacion sobre edad y estudios, etc.
# Los que estan en postulaciones pero NO en gye y edu, fueron descartados en este paso.
postulantes_filtrados_efectivos_df.head()

,idpostulante,sexo,nombre,estado,edad
0,NjlD,1,7,2,37
1,ZaO5,1,6,2,40
2,NdJl,2,4,1,49
3,5kNq,2,3,1,45
4,Ez8J,2,7,0,50


In [228]:
postulantes_filtrados_efectivos_df.shape

(348864, 5)

# Armado del Set de Entrenamiento

In [306]:
# Primero armo un dataframe con la informacion de postulaciones que ya tengo -> postulantes_df
# Observar que aca puede haber postulantes sobre los cuales no tengo informacion sobre genero y edad.
# Tambien puede haber postulantes que luego no aparezcan en el test final.


# ¿Deberia filtrar los postulantes sobre los cuales no tengo info sobre su genero y edad?
avisos = avisos_detalle_df.drop_duplicates('idaviso')
avisos = avisos['idaviso'].to_frame()

postulantes = postulantes_filtrados_df.drop_duplicates('idpostulante')
postulantes = postulantes['idpostulante'].to_frame()


postulantes_df =  postulaciones_df.merge(avisos,how='inner',on='idaviso')
postulantes_df = postulantes_df.merge(postulantes,how='inner',on='idpostulante')

postulantes_df =postulantes_df.groupby('idpostulante').agg({'idaviso':'value_counts'})
postulantes_df.columns=['postulacion']
postulantes_df['postulacion'] = 1 #Para los casos en que se repiten.

avisos = 0
postulantes = 0

In [307]:
postulantes_df.head()

postulacion
idpostulante idaviso                
0z5Dmrd      1112384041            1
             1112420060            1
0z5JW1r      1112257171            1
             1112281879            1
             1112292169            1

In [308]:
postulantes_df.reset_index(inplace=True)

In [309]:
postulantes_df.head()

,idpostulante,idaviso,postulacion
0,0z5Dmrd,1112384041,1
1,0z5Dmrd,1112420060,1
2,0z5JW1r,1112257171,1
3,0z5JW1r,1112281879,1
4,0z5JW1r,1112292169,1


In [310]:
postulantes_df.idaviso.unique().shape

(18695,)

In [311]:
avisos_detalle_df['idaviso'].unique().shape

(25288,)

In [312]:
#Armado de NO postulaciones

#Verificar que las listas utilizadas sean las correctas. Estamos usando solo avisos con postulacion.

#¿Podriamos usar los avisos que no tienen postulacion?


lista = []
user_list = postulantes_filtrados_df['idpostulante'].unique() #Utilizo a todos los usuarios. 
aviso_list = avisos_detalle_df['idaviso'].unique() #Utilizo todos los avisos.
index = 1
for n in range(10000):
    for i in range(500):
        user_index = random.randint(0,504371)
        aviso_index = random.randint(0,25287)
        lista.append((index,user_list[user_index],aviso_list[aviso_index],0))
        index+=1
        

In [313]:
no_postulantes_df = pd.DataFrame(lista)

In [314]:
#postulantes_filtrados_df=0
#avisos_detalle_df= 0
#avisos_detalle_filtrados_df=0
#avisos_online_df=0
#postulaciones_df=0
lista=0

In [315]:
no_postulantes_df.memory_usage(deep=True).sum()

438943469

In [316]:
no_postulantes_df.head()

,0,1,2,3
0,1,xkPJvDP,1761070,0
1,2,0zB6NRY,1112277179,0
2,3,19GkAb,1112313343,0
3,4,xkvEzjV,1111190024,0
4,5,RzMJDkO,1112121058,0


In [317]:
no_postulantes_df.drop(0,axis='columns',inplace=True)

In [318]:
no_postulantes_df.head()

,1,2,3
0,xkPJvDP,1761070,0
1,0zB6NRY,1112277179,0
2,19GkAb,1112313343,0
3,xkvEzjV,1111190024,0
4,RzMJDkO,1112121058,0


In [319]:
no_postulantes_df.columns= ['idpostulante','idaviso','postulacion']

In [320]:
no_postulantes_df.shape

(5000000, 3)

In [321]:
no_postulantes_df.head()

,idpostulante,idaviso,postulacion
0,xkPJvDP,1761070,0
1,0zB6NRY,1112277179,0
2,19GkAb,1112313343,0
3,xkvEzjV,1111190024,0
4,RzMJDkO,1112121058,0


In [322]:
postulantes_df.head()

,idpostulante,idaviso,postulacion
0,0z5Dmrd,1112384041,1
1,0z5Dmrd,1112420060,1
2,0z5JW1r,1112257171,1
3,0z5JW1r,1112281879,1
4,0z5JW1r,1112292169,1


In [323]:
#Junto el dataframe donde tengo los postulantes con el aviso al que se postularon
#Con el dataframe generado al azar.
entrenamiento_df = no_postulantes_df.merge(postulantes_df,how='outer',on=['idpostulante','idaviso'])

In [324]:
entrenamiento_df.head()

,idpostulante,idaviso,postulacion_x,postulacion_y
0,xkPJvDP,1761070,0.0,NaN
1,0zB6NRY,1112277179,0.0,NaN
2,19GkAb,1112313343,0.0,NaN
3,xkvEzjV,1111190024,0.0,NaN
4,RzMJDkO,1112121058,0.0,NaN


In [325]:
entrenamiento_df['postulacion_x'].fillna(0,inplace=True)
entrenamiento_df['postulacion_y'].fillna(0,inplace=True)

In [326]:
entrenamiento_df['postulacion'] = entrenamiento_df['postulacion_x'] + entrenamiento_df['postulacion_y']

In [327]:
entrenamiento_df.head()

,idpostulante,idaviso,postulacion_x,postulacion_y,postulacion
0,xkPJvDP,1761070,0.0,0.0,0.0
1,0zB6NRY,1112277179,0.0,0.0,0.0
2,19GkAb,1112313343,0.0,0.0,0.0
3,xkvEzjV,1111190024,0.0,0.0,0.0
4,RzMJDkO,1112121058,0.0,0.0,0.0


In [328]:
 entrenamiento_df.drop(['postulacion_x','postulacion_y'],axis='columns',inplace=True)

In [329]:
entrenamiento_df['postulacion'] = pd.Categorical(entrenamiento_df['postulacion'])
entrenamiento_df['postulacion'] = entrenamiento_df['postulacion'].cat.codes
entrenamiento_df['postulacion'].value_counts()

1    6187829
0    4997584
Name: postulacion, dtype: int64

In [330]:
# Hasta aca Ya tenemos nuestro formato (id,postulante,aviso,postulacion)
entrenamiento_df.head()

,idpostulante,idaviso,postulacion
0,xkPJvDP,1761070,0
1,0zB6NRY,1112277179,0
2,19GkAb,1112313343,0
3,xkvEzjV,1111190024,0
4,RzMJDkO,1112121058,0


In [331]:
entrenamiento_df.tail()

,idpostulante,idaviso,postulacion
11185408,zvxzrWm,1112438823,1
11185409,zvxzrWm,1112442213,1
11185410,zvxzrWm,1112445486,1
11185411,zvxzrWm,1112448822,1
11185412,zvxzrWm,1112455515,1


In [332]:
entrenamiento_df.idaviso.unique().shape

(25288,)

In [333]:
avisos_detalle_df.idaviso.unique().shape

(25288,)

# Armo la Matriz de Features

In [334]:
matrix_df = entrenamiento_df.merge(avisos_detalle_df,how='inner',on='idaviso')

In [335]:
matrix_df.idaviso.shape

(11185413,)

In [336]:
#Deberian ser iguales.
matrix_df.shape[0] == entrenamiento_df.shape[0]

True

In [337]:
matrix_df = matrix_df.merge(postulantes_filtrados_df,how='inner',on='idpostulante')

In [338]:
#Deberian ser iguales.
matrix_df.shape[0] == entrenamiento_df.shape[0]

True

In [339]:
#Me quedo con el indice como "id" de la tupla postulante-aviso.
matrix_df.drop(columns=['idpostulante','idaviso'],axis=1,inplace=True)

In [340]:
matrix_df.head()

,postulacion,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,sexo,nombre,estado,edad
0,0,1,Project Engineer,"Importante empresa multinacional, seleccionará...",Capital Federal,NaN,NaN,Full-time,Otro,Tecnologia / Sistemas,Experis,2,7,1,25
1,1,1,Representante de atención telefónica con perfi...,<p>Importante empresa del rubro telecomunicaci...,Gran Buenos Aires,NaN,NaN,Part-time,Junior,Comercial,ACC Group,2,7,1,25
2,0,1,REPRESENTANTE DE TRADE MARKETING -Canal Farmac...,<p>Estamos buscando Representantes de <strong>...,Gran Buenos Aires,NaN,NaN,Full-time,Otro,Marketing,Proven Group,2,7,1,25
3,1,1,Atención al Cliente Corporativo Z/ Capital Fed...,<p>Para importante compañía del rubro logístic...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,Assistem,2,7,1,25
4,1,1,Operador telefónico - Atención al Cliente,<p>Importante Empresa se encuentra en la búsqu...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Call Center,M-LAT S.A.,2,7,1,25


# Me fijo si las vistas implican una no postulacion

In [341]:
vistas_df.head()

,idAviso,timestamp,idpostulante
0,1112400775,2018-04-06T01:43:47.445-0400,kPjr2K0
1,1112400775,2018-04-06T01:43:48.330-0400,kPjr2K0
2,1112426664,2018-04-06T01:43:50.042-0400,GNOwZmb
3,1112426664,2018-04-06T01:43:51.171-0400,GNOwZmb
4,1112445738,2018-04-06T01:43:46.371-0400,lDLxajz


In [342]:
vistas_df.columns=['idaviso','timestamp','idpostulante']

In [343]:
postulaciones_df.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


In [344]:
#Efectivamente una "vista" no implica una no postulacion.
postulaciones_df.merge(vistas_df,on=['idaviso','idpostulante'],how='inner').head()

,idaviso,idpostulante,fechapostulacion,timestamp
0,1112364763,ZaO5,2018-02-27 20:54:50,2018-02-27T20:53:53.668-0500
1,1112383420,ZaO5,2018-03-07 08:16:46,2018-04-02T18:38:56.797-0400
2,1112383420,ZaO5,2018-03-07 08:16:46,2018-04-02T18:39:20.453-0400
3,1112397910,ZaO5,2018-03-18 15:35:40,2018-04-02T18:42:17.247-0400
4,1112397910,ZaO5,2018-03-18 15:35:40,2018-04-02T18:44:18.643-0400


# Test Final DF

In [ ]:
test_final_df = pd.read_csv('test_final_100k.csv')

In [30]:
test_final_df.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [31]:
test_final_df['idpostulante'].unique().shape

(64646,)

In [32]:
#Los postulantes que tengo informacion que aparecen en el test final.
#Entonces hay casi 20mil postulantes sobre los cuales no tengo informacion.
test_final_df.drop_duplicates('idpostulante').merge(postulantes_filtrados_df,on='idpostulante',how='inner').shape

(44930, 7)

# KNN

In [159]:
knn = KNeighborsClassifier(n_neighbors= 5)

In [160]:
#knn.fit(df_1, y)

In [161]:
test_final_df.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [79]:

#df_1.drop(columns=['idpostulante','idaviso'],axis=1,inplace=True)

#y = df_1['postulacion']
#df_1.drop(columns='postulacion',axis=1,inplace=True)
#df_1.head()


#test_final_df['sepostulo'] = 1
#test_final_df.drop(columns=['idaviso','idpostulante'],axis=1,inplace=True)
#test_final_df.head()